In [43]:
from natasha import NewsNERTagger
from natasha import MorphVocab, NewsEmbedding, NewsMorphTagger
from natasha import Doc, Segmenter
import json
import pandas as pd
import re

In [37]:
import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()

In [47]:
morph_tagger = NewsMorphTagger(embedding)
morph_vocab = MorphVocab()

In [29]:
with open("../data/corpus_as_dict.json") as f:
    docs = json.load(f)

In [35]:
titles = list(docs.keys())
texts = list(docs.values())
df = pd.DataFrame.from_dict({'title':titles, 'text':texts})

In [36]:
df.sample(5)

,title,text
141,Ледник,Ледник — масса льда преимущественно атмосферно...
78,Дифферент,"Дифферент — морской термин, разница осадок суд..."
268,Скакалка,Скакалка — спортивный снаряд для физических уп...
72,Двойное дно,"Двойное дно — кораблестроительный термин, част..."
300,Титаник. Легенда продолжается,«Титаник. Легенда продолжается» — комедийный с...


In [45]:
def get_ner(transcript):
  script = Doc(re.sub(r'\((.*?)\)', "", transcript))
  script.segment(segmenter)
  script.tag_morph(morph_tagger)
  for token in script.tokens:
    token.lemmatize(morph_vocab)
  script.tag_ner(ner_tagger)
  for span in script.spans:
    span.normalize(morph_vocab)
  named_ents = [(i.text, i.type, i.normal) for i in script.spans]
  normed_ents = []
  for word, tag, norm in named_ents:
    if len(word.split()) == 1 and tag == "LOC":
      for gram in range(len(analyzer.parse(word))):
        if "Geox" in analyzer.parse(word)[gram].tag:
          normed_ents.append((analyzer.parse(word)[gram].normal_form))
          break
        elif gram == len(analyzer.parse(word)) - 1:
          normed_ents.append((norm.lower().strip(".,!?;-")))
    else:
      normed_ents.append((norm.lower().strip(".,!?;-")))
  return sorted(normed_ents)

In [54]:
df["named_entities"] = df.apply(lambda row: get_ner(row["text"]), axis=1)

In [55]:
df.sample(3)

,title,text,named_entities
27,Бифштекс,"Бифштекс, стейк-филе — блюдо из жареной говяди...","[бифштекс, британия, сша, сша, сша]"
159,Машинный телеграф,Машинный телеграф — устройство для передачи из...,[]
316,Углекислый газ,Диоксид углерода или двуокись углерода — бесцв...,"[земля, земля, земля, земля, земля, земля, сол..."


In [58]:
has_ner = [i for i in df.index.values if df.named_entities[i]]

In [59]:
len(has_ner), df.shape[0]

(223, 363)

In [60]:
df_ner = df[df.index.isin(has_ner)]

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

In [62]:
ner_voc = []
for row in df_ner.named_entities.tolist():
  ner_voc.extend(row)
len(ner_voc), len(set(ner_voc))

(1758, 955)

In [63]:
vocabulary = sorted(set(ner_voc))
corpus = df_ner.named_entities.apply(str).tolist()

In [64]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe.fit_transform(corpus)
km = KMeans(n_clusters=30, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)

In [65]:
km.fit(X)

/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:786: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  warnings.warn("'precompute_distances' was deprecated in version "


KMeans(algorithm='full', max_iter=600, n_clusters=30, precompute_distances=True)

In [67]:
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.12205426569682176
1.2836337163552074


In [68]:
df_ner["label"] = km.predict(X)

<ipython-input-68-20dc8af91091>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ner["label"] = km.predict(X)


In [69]:
df_ner["label"].value_counts()

2     151
6       9
1       8
16      7
5       4
23      3
8       3
9       3
13      3
21      2
20      2
19      2
18      2
17      2
15      2
10      2
7       2
4       2
3       2
27      2
26      1
24      1
28      1
25      1
0       1
22      1
14      1
12      1
11      1
29      1
Name: label, dtype: int64

In [71]:
df_ner.query("label == 8")

,title,text,named_entities,label
52,Вокзал Сен-Лазар,Вокзал Сен-Лазар — одна из шести крупных голов...,"[европа, иль-де-франс, париж, сен-лазар, сен-л...",8
186,Национальная библиотека Франции,Национальная библиотека Франции — библиотека в...,"[европа, национальная библиотека, париж, париж...",8
205,Париж,"Париж — столица и крупнейший город Франции, а ...","[большой париж, версаль, евросоюз, иль-де-фран...",8


In [72]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(20):
  print("Cluster %d:" % i, end='')
  for ind in order_centroids[i, :10]:
    print(' %s' % terms[ind], end='')
  print()

Cluster 0: бангор белфаст керрикфергус керрикфергусский холивуд антрим даун ирландия хердман келтроу
Cluster 1: сша конгресс америка nautilus вмс штат бифштекс британия bloomberg swift
Cluster 2: европа канада сша азия атлантический россия англия великобритания северная австралия
Cluster 3: атлантика северная азовское евразия шпицберген черное америка атлантический жоау фернандеша лэврадур жан кальвин
Cluster 4: пасленовые паслен японское море ж. б. ламарком залив белфаст зал слава закавказье жюль мишле жюль ардуэн-мансар жоау фернандеша лэврадур
Cluster 5: россия российская англия сша эстония ленинград нева снг кронштадт эрмитаж
Cluster 6: ирландия великобритания англия лондон северная шотландия уэльс уайтхолл европа белфаст
Cluster 7: америка индия северная гренландия земля бермуды микелон австралия канада сша
Cluster 8: париж франция сена европа ришелье паризии сите евросоюз версаль юнеско
Cluster 9: испания мадрид европа африка португалия андорра марокко канарские гибралтар нато
Cl